In [4]:
import pandas as pd
import numpy as np
import os

#import datetime as dt
from datetime import datetime
import pycountry as pc
import category_encoders as ce

# for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D

# for moths
from feature_engine.creation import CyclicalFeatures

# for clusters
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [5]:
#### important: set train end year (e.g. if 2017, then test data starts from Jan 2018)

train_end = 2017
unit_of_analyis = 'isocode'

## Import and join data
We import the merged data set containing the target (deaths) and the covariates (GDELT event counts) alongside the World Bank's population data.
- We ensure both data sets start from 1989. 
- We ensure both data sets contain the same countries. As part of this process we eliminate several countries that rarely appear in GDELT.
- Missing values in the population: for 2022 and 2023 we use the population of 2021, only Palestine is missing data for 1989, so we impute the 1990 population

In [6]:
path = os.getcwd()
parent_path = os.path.abspath(os.path.join(path, os.pardir))

print('parent_path: ', parent_path)

merged_og = pd.read_csv(os.path.abspath(parent_path + '/data/merged.csv'))
wb_pop_og = pd.read_csv(os.path.abspath(parent_path + '/data/WorldBank_pop/wb_pop.csv'), sep=';') # NB in wide format
wb_pop_og.drop(columns=['Unnamed: 67', '2022', 'Indicator Name', 'Indicator Code'], inplace=True)
extra_pop = pd.read_csv(os.path.abspath(parent_path + '/data/TWN_ESH.csv'), sep=',') # We add population estimates for Taiwan and Western Sahara from a separate source

merged_og = merged_og[merged_og['year'] >= 1989]
extra_pop = extra_pop[extra_pop['year'] >= 1989]

print('countries with pop data: ',wb_pop_og['Country Code'].nunique())
print('countries in merged data: ',merged_og['isocode'].nunique())
print('extra pop countries: ',extra_pop['isocode'].nunique())


parent_path:  /Users/giovannachaves/Documents/BSE/Master's Thesis/thesis


/Users/giovannachaves/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (73,74) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


FileNotFoundError: [Errno 2] No such file or directory: "/Users/giovannachaves/Documents/BSE/Master's Thesis/thesis/data/WorldBank_pop/wb_pop.csv"

In [ ]:
merged_og.loc[merged_og['isocode'] == 'MDV', ['year', 'month']]

In [ ]:
merged = merged_og.copy()

# Because of issue with MonthYear column: create new column with year and month
merged['year'] = merged['year'].astype(str)
merged['month'] = merged['month'].astype(str)

# Pad the month column with leading zeros if needed (e.g., convert '1' to '01')
merged['month'] = merged['month'].str.zfill(2)

# Concatenate the year and month columns with a '-' &  convert to datetime object
merged['month_year'] = merged['year'] + '-' + merged['month']
merged['month_year'] = pd.to_datetime(merged['month_year'], format='%Y-%m')


# compare month_year and MonthYear
merged.MonthYear = pd.to_datetime(merged.MonthYear)

print('entires merged:', len(merged))
print('date range:', merged.month_year.min(), 'to', merged.month_year.max())
# print('date range:', merged.MonthYear.min(), 'to', merged.MonthYear.max())

# merged[['month_year', 'MonthYear']].head(3)

In [ ]:
# Check which countries are in the GDELT-UCDP dataset but not in the World Bank population or the extra pop dataset
missing_pop_data = set(merged['isocode'].unique()) - set(wb_pop_og['Country Code'].unique()) - set(extra_pop['isocode'].unique())

print('Countries without population data (pre drop):', len(missing_pop_data))
for isoc in missing_pop_data:
    print(pc.countries.get(alpha_3=isoc).name)

# Drop rarely mentioned countries (with with <413 Month Entries in GDELT)
# exception: 'SSD'
isocodes_to_drop = ['ABW', 'AIA', 'AND', 'ASM', 'BVT', 'COK', 'CXR', 'CYM', 'FLK', 'FRO', 'GGY', 'GLP', 'GUF', 'HMD', 'IMN', 'IOT', 'KIR', 'KNA', 'LCA', 'LIE', 'MTQ', 'MYT', 'NFK', 'NIU', 'NRU', 'PCN', 'PLW', 'PYF', 'REU', 'SHN', 'SJM', 'SMR', 'SPM', 'TCA', 'TKL', 'TUV', 'VCT', 'VGB', 'WLF']
merged = merged[~merged['isocode'].isin(isocodes_to_drop)]

# Check missing counries again
missing_pop_data = set(merged['isocode'].unique()) - set(wb_pop_og['Country Code'].unique()) - set(extra_pop['isocode'].unique())

print('___________________________________________________________')
print('Countries without population data (post drop):', len(missing_pop_data))

for isoc in missing_pop_data:
    print(pc.countries.get(alpha_3=isoc).name)

In [ ]:
# Drop the remaining countries without population data - no longer having to drop Western Sahara and Taiwan
isocodes_to_drop = ['ATA', 'JEY', 'MSR', 'VAT'] #'ESH' 'TWN'
merged = merged[~merged['isocode'].isin(isocodes_to_drop)]

In [ ]:
# After March 2023, no deaths present - so assume not matched to UCDP data and not used
for date in ['2023-02-01','2023-03-01', '2023-04-01', '2023-05-01']:
    print(f'countries: in {date}: ', len(merged[merged['month_year'] == date]['deaths']))
    print('of which nan deaths:      ',merged[merged['month_year'] == date]['deaths'].isnull().sum())

## Impute rows for missing months
We want a complete set of months between the first and the final mention of a country in our dataset. At the end we verify that we have the expected number of months for each country.



In [ ]:
#df_fill = df_merged.copy()
df_fill = merged.copy() 

In [7]:
# 'year' and 'month' columns to a datetime type
df_fill['date'] = pd.to_datetime(df_fill[['year', 'month']].assign(day=1))

# columns to be filled
fill_cols = [col for col in df_fill.columns if 'events' in col] # previously 'counts', but that missed teh normalised total event

dfs = []

# Iterate through unique 'isocode' values
for isocode in df_fill['isocode'].unique():
    df_isocode = df_fill[df_fill['isocode'] == isocode].copy()

    # Create a new dataframe that covers all months between the min and max 'Date' for this isocode
    min_date = df_isocode['date'].min()
    max_date = df_isocode['date'].max()
    all_dates = pd.DataFrame(pd.date_range(min_date, max_date, freq='MS'), columns=['date'])

    # merge onto the existing dataframe
    df_isocode = pd.merge(all_dates, df_isocode, on='date', how='left')

    df_isocode['isocode'] = isocode

    # forward-fill and then fill any remaining NaNs with 0
    df_isocode[fill_cols] = df_isocode[fill_cols].ffill().fillna(0)

    # convert the 'date' back to 'year' and 'month'
    df_isocode['year'] = df_isocode['date'].dt.year
    df_isocode['month'] = df_isocode['date'].dt.month

    # append to list
    dfs.append(df_isocode)

# concatenate all DataFrames in the list into a final DataFrame
df_filled = pd.concat(dfs)

#df_final.drop(columns=['date'], inplace=True)

# sort final dataframe by 'isocode', 'year' and 'month'
df_filled.sort_values(['year', 'month', 'isocode'], inplace=True)
df_filled.fillna(0)

df_filled.drop(columns=['MonthYear'], inplace=True)

NameError: name 'df_fill' is not defined

In [8]:
def get_country_name(iso_code):
    try:
        return pc.countries.get(alpha_3=iso_code).name
    except AttributeError:
        return None

df_filled['country'] = df_filled['isocode'].apply(get_country_name)

NameError: name 'df_filled' is not defined

In [9]:
def check_month_year_sequence(group):
    # Create the expected sequence of months and years
    min_year, min_month = group[['year', 'month']].iloc[0] # use the first row of each group
    max_year, max_month = group[['year', 'month']].iloc[-1] # use the last row of each group

    expected_month_years = [(y, m) for y in range(min_year, max_year + 1) for m in range(1, 13)]
    
    # If there is only one year in the data, filter for months within the min and max range
    if min_year == max_year:
        expected_month_years = [my for my in expected_month_years if min_month <= my[1] <= max_month]
    else:
        # If there are multiple years, adjust for the first and last years
        expected_month_years = [my for my in expected_month_years if 
                                not (my[0] == min_year and my[1] < min_month) and 
                                not (my[0] == max_year and my[1] > max_month)]
                                
    # Check if the sequence of month-years in the group is equal to the expected sequence
    actual_month_years = sorted(list(zip(group['year'], group['month'])))
    
    if actual_month_years != expected_month_years:
        print(f"Incorrect sequence for isocode: {group['isocode'].iloc[0]}")
        print(f"Expected: {expected_month_years}")
        print(f"Actual: {actual_month_years}")
        
    return actual_month_years == expected_month_years


In [10]:
# Apply the function to each group
is_sequence_correct = df_filled.groupby('isocode').apply(check_month_year_sequence)

# Check if the sequence of month-years is correct for all isocodes
assert is_sequence_correct.all(), "The sequence of month-years is not correct for some isocodes"

# check how many rows added
print('Rows added:', len(df_filled) - len(df_fill))

NameError: name 'df_filled' is not defined

## Population data 

### Melt WB data into long format 
Fill 2022 and 2023 with 2021 population value and cut off at 1989.

In [11]:
pop = wb_pop_og.copy()
# fill missing complete years (in future to be replaces on basis of growth rate)
pop['2022'] = pop['2021']
pop['2023'] = pop['2021']
pop.head(3)

NameError: name 'wb_pop_og' is not defined

In [12]:
# We realised that Palestine is missing population data for 1989 and impute it with 1990 values
missing_1989 = pop[pop['1989'].isnull()]
missing_1989['1989'] = missing_1989['1990']

# Update the original DataFrame with the imputed values
pop.update(missing_1989)
#pop[pop['1989'].isnull()] # only INX which doesn't seem to be a country
county_to_drop = ['INX']
pop = pop[~pop['Country Code'].isin(county_to_drop)]

# melt population data
pop = pd.melt(pop, id_vars=('Country Code', 'Country Name'), var_name='Year', value_name='wb_pop')

pop.rename(columns={'Country Code': 'isocode'}, inplace=True)
pop.rename(columns={'Year': 'year'}, inplace=True)
pop = pop[pop['year'] >= '1989']


# checks
print(pop.year.min(), pop.year.max())
pop.isnull().sum()

NameError: name 'pop' is not defined

### Interpolate missing years for extra population data
Western Sahara and Taiwan

In [ ]:
all_years = np.arange(1989, 2024)

extra_pop_all = pd.DataFrame()
for isocode in extra_pop['isocode'].unique():
    # Create a new data frame with all possible years
    complete_group = pd.DataFrame({'year': all_years, 'isocode': isocode})
    # Merge the original data frame with the complete group based on isocode and year
    merged_group = pd.merge(complete_group, extra_pop, on=['isocode', 'year'], how='left')
    # Perform linear interpolation within the group
    merged_group['population'] = merged_group['population'].interpolate(method='linear')
    # Forward fill and backward fill any remaining missing values
    merged_group['population'] = merged_group['population'].ffill().bfill()
    # Add the complete group to the final data frame
    extra_pop_all = pd.concat([extra_pop_all, merged_group])

# Sort the complete data frame by isocode and year
extra_pop_all.sort_values(['isocode', 'year'], inplace=True)

extra_pop_all.rename(columns={'population': 'wb_pop'}, inplace=True)

extra_pop_all.drop(columns=['Country Name'], inplace=True)
print('Rows added:', len(extra_pop_all) - len(extra_pop))
print(extra_pop_all.year.min(), extra_pop_all.year.max())
extra_pop_all.isnull().sum()

### Visualisations
Shows that population distributions change depending on which years are taken into account. Although year-on-year percentage changes are close to zero for most countries.

In [ ]:
early_train_start = 1989
late_train_start = 2000
train_end = 2017
test_end = 2022

selected_isocodes = ["AFG", "ARE", "BDI", "BHR", "BIH", "CYM", "DJI", "ERI", "GNQ", "ISR", "JOR", "KHM", "KWT", "LBN", "LBR", "LBY", "MAF", "MNP", "NAM", "OMN", "QAT", "RWA", "SGP", "SLE", "SOM", "SSD", "SXM", "SYR", "TCA", "XKX"]

# "AFG", "ARE", "BDI", "BHR", "BIH", "CYM", "DJI", "ERI", "GNQ", "ISR", "JOR", "KHM", "KWT", "LBN", "LBR", "LBY", "MAF", "MNP", "NAM", "OMN", "QAT", "RWA", "SGP", "SLE", "SOM", "SSD", "SXM", "SYR", "TCA", "XKX"
filtered_data = pop[pop['isocode'].isin(selected_isocodes)]


# Filter the data for the two time periods: 1989 vs 2000 until end of train 
filtered_data['year'] = filtered_data['year'].astype(int)

data_1989_train_end = filtered_data[(filtered_data['year'] >= 1989) & (filtered_data['year'] <= train_end)]
#data_2000_2022 = filtered_data[(filtered_data['year'] >= 2000) & (filtered_data['year'] <= 2022)]
data_2000_train_end = filtered_data[(filtered_data['year'] >= 2000) & (filtered_data['year'] <= train_end)]

# Create a list to store the labels for each country
labels = []
colors = []
data_combined = []
for isocode in selected_isocodes:
    labels.append(isocode)
    labels.append('')
    colors.extend(['red', 'blue']) #'green'
    data_combined.append(data_1989_train_end[data_1989_train_end['isocode'] == isocode]['wb_pop'])
    #data_combined.append(data_2000_2022[data_2000_2022['isocode'] == isocode]['wb_pop'])
    data_combined.append(data_2000_train_end[data_2000_train_end['isocode'] == isocode]['wb_pop'])

# Create the boxplot using matplotlib
plt.figure(figsize=(10, 6))
plt.boxplot(data_combined, labels=labels, patch_artist=True,
            boxprops=dict(facecolor='white', color='black'),
            capprops=dict(color='black'), whiskerprops=dict(color='black'),
            flierprops=dict(color='black', markeredgecolor='black'), medianprops=dict(color='black'))

for patch, color in zip(plt.boxplot(data_combined, patch_artist=True)['boxes'], colors):
    patch.set_facecolor(color)
    
# Create a legend for the time period colors
legend_elements = [plt.Rectangle((0, 0), 1, 1, color='red', label=f'1989-{train_end}'),
                   #plt.Rectangle((0, 0), 1, 1, color='blue', label='2000-2022'),
                   plt.Rectangle((0, 0), 1, 1, color='blue', label=f'2000-{train_end}')
                   ]
plt.legend(handles=legend_elements, loc='upper right')

plt.title('World Bank Population distributions in different time periods')
plt.xlabel('Country')
plt.ylabel('wb_pop')
plt.grid(color='grey', linestyle='--', linewidth=0.1)
plt.xticks(np.arange(1, len(selected_isocodes) * 2, 2), selected_isocodes, rotation= 45)
plt.show()

In [ ]:
early_train_start = 1989
late_train_start = 2000
train_end = 2017
test_end = 2023

selected_isocodes = ["AFG", "ARE", "BDI", "BHR", "BIH", "CYM", "DJI", "ERI", "GNQ", "ISR", "JOR", "KHM", "KWT", "LBN", "LBR", "LBY", "MAF", "MNP", "NAM", "OMN", "QAT", "RWA", "SGP", "SLE", "SOM", "SSD", "SXM", "SYR", "TCA", "XKX"]

# "AFG", "ARE", "BDI", "BHR", "BIH", "CYM", "DJI", "ERI", "GNQ", "ISR", "JOR", "KHM", "KWT", "LBN", "LBR", "LBY", "MAF", "MNP", "NAM", "OMN", "QAT", "RWA", "SGP", "SLE", "SOM", "SSD", "SXM", "SYR", "TCA", "XKX"
filtered_data = pop[pop['isocode'].isin(selected_isocodes)]


# Filter the data for the two time periods: 1989 vs 2000 until end of train 
filtered_data['year'] = filtered_data['year'].astype(int)

# 2000-2017
data_pop_1 = filtered_data[(filtered_data['year'] >= 2000) & (filtered_data['year'] <= train_end)]
#data_2000_2022 = filtered_data[(filtered_data['year'] >= 2000) & (filtered_data['year'] <= 2022)]

# 2000-2023
data_pop_2 = filtered_data[(filtered_data['year'] >= 2000) & (filtered_data['year'] <= test_end)]

# Create a list to store the labels for each country
labels = []
colors = []
data_combined = []
for isocode in selected_isocodes:
    labels.append(isocode)
    labels.append('')
    colors.extend(['red', 'blue']) #'green'
    data_combined.append(data_pop_1[data_pop_1['isocode'] == isocode]['wb_pop'])
    data_combined.append(data_pop_2[data_pop_2['isocode'] == isocode]['wb_pop'])

# Create the boxplot using matplotlib
plt.figure(figsize=(10, 6))
plt.boxplot(data_combined, labels=labels, patch_artist=True,
            boxprops=dict(facecolor='white', color='black'),
            capprops=dict(color='black'), whiskerprops=dict(color='black'),
            flierprops=dict(color='black', markeredgecolor='black'), medianprops=dict(color='black'))

for patch, color in zip(plt.boxplot(data_combined, patch_artist=True)['boxes'], colors):
    patch.set_facecolor(color)
    
# Create a legend for the time period colors
legend_elements = [plt.Rectangle((0, 0), 1, 1, color='red', label=f'{late_train_start}-{train_end}'),
                   #plt.Rectangle((0, 0), 1, 1, color='blue', label='2000-2022'),
                   plt.Rectangle((0, 0), 1, 1, color='blue', label=f'{train_end}-{test_end}')
                   ]
plt.legend(handles=legend_elements, loc='upper right')

plt.title('World Bank Population distributions in different time periods') #with a percentage change of more than 5%
plt.xlabel('Country')
plt.ylabel('wb_pop')
plt.grid(color='grey', linestyle='--', linewidth=0.1)
plt.xticks(np.arange(1, len(selected_isocodes) * 2, 2), selected_isocodes, rotation= 45)
plt.show()

In [ ]:
# Calculate the year-on-year percentage change for each isocode
pop['pct_change'] = pop.groupby('isocode')['wb_pop'].pct_change() * 100

# Create a separate line plot for each country
fig, ax = plt.subplots(figsize=(10, 6))
for isocode, data in pop.groupby('isocode'):
    ax.plot(data['year'], data['pct_change'], label=isocode)

# Set the legend outside the graph
#ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Set the axis labels and title
ax.set_xlabel('Year')
ax.set_ylabel('Percentage Change in wb_pop')
ax.set_title('Year-on-Year Percentage Change in wb_pop for All Isocodes')
plt.xticks(np.arange(1, pop.year.nunique()), rotation=45)

pop.drop(columns=['pct_change'], inplace=True)

# Display the plot
plt.show()


### Option 1: work with averages
IMPORTANT: Decide when training data ends!

In [ ]:
train_end = 2017

In [ ]:
print(pop.isnull().sum().sum())

In [ ]:
pop['year'] = pop['year'].astype(int)

# Filter the dataframe for the desired years (2020-train_end) / (1989-train_end))
pop_2000_to_train_end = pop[pop['year'].between(2000, train_end)]
pop_1989_to_train_end = pop[pop['year'].between(1989, train_end)]

# Group by 'isocode' and 'Country Name' columns and calculate the average population
grouped_2000_to_train_end = pop_2000_to_train_end.groupby(['isocode', 'Country Name'])['wb_pop'].mean().reset_index()
grouped_1989_to_train_end = pop_1989_to_train_end.groupby(['isocode', 'Country Name'])['wb_pop'].mean().reset_index()

# Merge & rename the columns
pop_avrg = pd.merge(grouped_1989_to_train_end, grouped_2000_to_train_end, on=['isocode', 'Country Name'], suffixes=(f'_1989_{train_end}', f'_2000_{train_end}'), how = 'left')
pop_avrg.rename(columns={f'wb_pop_2000_{train_end}': f'av_pop_2000_{train_end}', f'wb_pop_1989_{train_end}': f'av_pop_1989_{train_end}'}, inplace=True)

########
# same for taiwan and western sahara
extra_pop_all['year'] = extra_pop_all['year'].astype(int)

# Filter the dataframe for the desired years (2020-train_end) / (1989-train_end))
extra_pop_2000_to_train_end = extra_pop[extra_pop['year'].between(2000, train_end)]
extra_pop_1989_to_train_end = extra_pop[extra_pop['year'].between(1989, train_end)]

# Group by 'isocode' and 'Country Name' columns and calculate the average population
grouped_2000_to_train_end = extra_pop_2000_to_train_end.groupby(['isocode', 'Country Name'])['population'].mean().reset_index()
grouped_1989_to_train_end = extra_pop_1989_to_train_end.groupby(['isocode', 'Country Name'])['population'].mean().reset_index()

# Merge & rename the columns
extra_pop_avrg = pd.merge(grouped_1989_to_train_end, grouped_2000_to_train_end, on=['isocode', 'Country Name'], suffixes=(f'_1989_{train_end}', f'_2000_{train_end}'))
extra_pop_avrg.rename(columns={f'population_2000_{train_end}': f'av_pop_2000_{train_end}', f'population_1989_{train_end}': f'av_pop_1989_{train_end}'}, inplace=True)


### join them 
pop_avrg_all = pd.concat([pop_avrg, extra_pop_avrg], ignore_index=True)
pop_avrg_all

In [ ]:
# Merge population data with GDELT-UCDP data
df_merged_av_pop = pd.merge(merged, pop_avrg_all, on=['isocode'], how='left')

#df_merged_av_pop.drop(columns=['country'], inplace=True)
df_merged_av_pop['year'] = df_merged_av_pop['year'].astype(int)
df_pop_average = df_merged_av_pop[df_merged_av_pop['year'] >= 2000]

### Option 2: work with imputed changes


In [ ]:
pop2 = pop.copy()
xpop2 = extra_pop_all.copy()

In [ ]:
# Join extra data with world bank data
all_pop = pd.concat([pop2, xpop2], ignore_index=True)
all_pop.sort_values(['year', 'isocode'], inplace=True)

# Convert year column to datetime &  set the month and day to 1st of January
all_pop['month_year'] = pd.to_datetime(all_pop['year'], format='%Y')
all_pop['month_year'] = all_pop['month_year'].dt.strftime('%Y-%m-%d')
all_pop

In [ ]:
merged['month_year'] = pd.to_datetime(merged['month_year'], format='%Y-%m-%d')
merged['month_year'] = merged['month_year'].dt.strftime('%Y-%m-%d')
all_pop.drop(columns=['year'], inplace=True)
trial = pd.merge(merged, all_pop, on=['isocode', 'month_year'], how='left')

In [ ]:
trial.loc[trial['isocode'] == 'TWN']

In [ ]:
# Convert 'month_year' column to datetime type
trial['month_year'] = pd.to_datetime(trial['month_year'])

# Set 'isocode' and 'month_year' as the index
trial = trial.set_index(['isocode', 'month_year'])

# Function to interpolate missing values within a group
def interpolate_group(group):
    group['wb_pop'] = group['wb_pop'].interpolate(method='linear')
    return group

# Apply interpolation within each country group
trial = trial.groupby('isocode').apply(interpolate_group)

# Reset the index
trial = trial.reset_index(drop=False)

# # Filter the DataFrame to include only the desired range of months
# start_date = pd.to_datetime('2020-01-01')
# end_date = pd.to_datetime('2022-12-31')
# trial = trial[(trial['month_year'] >= start_date) & (trial['month_year'] <= end_date)]


In [ ]:
# NB from Jan 2021 onwards there will be no more changes as 2021 is the final year we have changing population data for
trial.loc[trial['isocode'] == 'AFG', ['month_year', 'wb_pop']][-55:-22]

In [ ]:
print('number of missing population values:', trial['wb_pop'].isnull().sum())
print('all missing population values are before', trial[trial['wb_pop'].isnull()].year.max())
print('countries: ', trial[trial['wb_pop'].isnull()].isocode.unique())

trial.drop(columns=['country'], inplace=True)


### SUBSET Januar 2000 to March 2023

In [ ]:
trial['year'] = trial['year'].astype(int)
df_pop_interpolated = trial[trial['year'] >= 2000]

df_pop_interpolated

### Join all population options in one dataframe

In [ ]:
# check dimensions are the same
print(df_pop_interpolated.shape, df_pop_average.shape)
print(df_pop_interpolated.isocode.nunique(), df_pop_average.isocode.nunique())

# have both options available
pop_options =  df_pop_average.copy()
pop_options['intp_pop'] =  df_pop_interpolated['wb_pop']
print('nulls:',pop_options.isnull().sum().sum(), '(mainly from deaths and event shares)')
pop_options.loc[pop_options['isocode'] == 'AFG', ['isocode', 'month_year', 'intp_pop', f'av_pop_2000_{train_end}', f'av_pop_1989_{train_end}']]


In [ ]:
# Only keep any years before 2023 and within 2023, only keep months that are not April or May
pop_options = pop_options[(pop_options['year'] != 2023) | ((pop_options['year'] == 2023) & (~pop_options['month'].isin(['04', '05'])))]

pop_options.month_year.max()

## Conflict definition(s)
Calculate deaths per 100,000 based on different population calculations.

Missing UCDP data: means no deaths for tracked dyads occured - assume zero casualties.

In [ ]:
df_conf = pop_options.copy()


In [ ]:
# fill missing deaths with 0
death_columns = ['deaths', 'state_deaths', 'nonstate_deaths', 'onesided_deaths', 'civilian_deaths']
df_conf[death_columns] = df_conf[death_columns].fillna(0)

In [ ]:
# generate deaths per 100,000 people for different population counts
population_versions = [f'av_pop_1989_{train_end}', f'av_pop_2000_{train_end}','intp_pop' ]

for wb_pop in population_versions:
    df_conf[f'deaths_all_{wb_pop}_pc'] = df_conf['deaths'] /  df_conf[wb_pop] *100_000
    #df_conf[f'deaths_state_{wb_pop}_pc'] = df_conf['state_deaths'] /  df_conf[wb_pop] *100_000
    

#df_conf['deaths_all_pc'] = df_conf['deaths'] /  df_conf['wb_pop'] *100_000
#df_conf['deaths_state_pc'] = df_conf['deaths'] /  df_conf['wb_pop'] *100_000

In [ ]:
# generate conflict dummy variables based on thresholds

for wb_pop in population_versions:
    df_conf[f'armedconf_{wb_pop}'] = df_conf[f'deaths_all_{wb_pop}_pc']>0.05 # more than 0.05 deaths per 100,000 people, i.e. 2 per 1 million 
    df_conf[f'civilwar_{wb_pop}'] = df_conf[f'deaths_all_{wb_pop}_pc']>3

# df_conf['armedconf'] = df_conf.deaths_all_pc>0.05
# df_conf['civilwar'] = df_conf.deaths_all_pc>3

In [ ]:
country_df = df_conf[df_conf['Country Name'] == 'Afghanistan']
country_df[['Country Name','month_year', f'av_pop_1989_{train_end}', f'av_pop_2000_{train_end}','intp_pop',
        f'deaths_all_av_pop_1989_{train_end}_pc', f'deaths_all_av_pop_2000_{train_end}_pc', 'deaths_all_intp_pop_pc',
       f'armedconf_av_pop_1989_{train_end}',  f'armedconf_av_pop_2000_{train_end}', 'armedconf_intp_pop',
       f'civilwar_av_pop_1989_{train_end}', f'civilwar_av_pop_2000_{train_end}', 'civilwar_intp_pop']]

## Additional targets

In [ ]:
df_targets = df_conf.copy()

In [ ]:
def calculate_escalation(df):
    df['lag_deaths'] = df.groupby('isocode')['deaths'].shift(1)

    df['delta_deaths'] = np.where((df['lag_deaths'] == 0) & (df['deaths'] == 0), 0,
                                  np.where((df['lag_deaths'] == 0) & (df['deaths'] != 0), np.inf,
                                           np.where((df['lag_deaths']).isna() == True, 0,
                                                    (df['deaths'] - df['lag_deaths']) / df['lag_deaths'])))

    # Group the data by 'isocode' and calculate the 75th percentile of the previous 24 months' delta_deaths
    df['threshold'] = df.groupby('isocode')['delta_deaths'].transform(lambda x: x.shift(1).rolling(window=24, min_periods=1).quantile(0.75))
    df['threshold'] = df['threshold'].fillna(0)

    # Check if the current month's delta_deaths exceeds the threshold or is infinity
    df['escalation'] = (df['deaths_choice'] >= 0.05) & ((df['delta_deaths'] > df['threshold']) | (df['delta_deaths'] == np.inf))
    df['escalation'] = df['escalation'].astype(int)

    return df

df_targets = calculate_escalation(df_targets)

df_targets.drop(columns=['lag_deaths', 'delta_deaths', 'threshold'], inplace=True)


# true_counts = df_targets['escalation'].sum()
# total_counts = df_targets['escalation'].count()
# percentage_true = (true_counts / total_counts) * 100

# summary_table = pd.DataFrame({'True Count': pd.Series(true_counts), 'Percentage True': pd.Series(percentage_true)})
# print(summary_table)

In [ ]:
column_list = [f'armedconf_av_pop_1989_{train_end}',  f'armedconf_av_pop_2000_{train_end}', 'armedconf_intp_pop',
       f'civilwar_av_pop_1989_{train_end}', f'civilwar_av_pop_2000_{train_end}', 'civilwar_intp_pop', 'escalation']

true_counts = df_targets[column_list].sum()
total_counts = df_targets[column_list].count()
percentage_true = (true_counts / total_counts) * 100

summary_table = pd.DataFrame({'True Count': true_counts , 'Percentage True': percentage_true}) #'Column': column_list, 
print(summary_table)

# in current preprocessing: df_og.armedconf.sum() = 7144

#print('Missing values:', df_targets.isna().sum().sum() ) #df_targets.loc[df_conf['isocode']=='TWN', f'armedconf_av_pop_1989_{train_end}'].sum())

## Past deaths and periods of peace
Important: decide which population and conflict definition to go with!
NB: Hannes used the total number of deaths rather than the deaths per capita


In [ ]:
df_past = df_targets.copy()

pop_choice = 'intp_pop'
deaths_choice = f'deaths_all_{pop_choice}_pc'
conflict_choice = f'armedconf_{pop_choice}'


### Drop everything that was not chosen

not_chosen_pops = [x for x in population_versions if x != pop_choice]

for not_picked in not_chosen_pops:
    print('dropping :', f'deaths_all_{not_picked}_pc', f'armedconf_{not_picked}', not_picked)
    df_past.drop(columns=[f'deaths_all_{not_picked}_pc', f'armedconf_{not_picked}', not_picked], inplace=True)

for vers in population_versions:
    print('dropping :', f'civilwar_{vers}')
    df_past.drop(columns=f'civilwar_{vers}', inplace=True)

In [ ]:
# adding columns for past deaths (rolling sum)

lcols = (df_past.groupby(unit_of_analyis)[deaths_choice] # for each country
         .transform(lambda x: x.rolling(y, min_periods=1).sum()) # rolling sum of best
         .rename('past' + str(y-1)) 
         for y in [7, 13, 61, 121]) 

df_past = df_past.join(pd.DataFrame(lcols).transpose())


In [ ]:
def count_peace(x):
    # number of periods in peace
    x = list(x) 
    y = []
    for n in range(0, len(x)):
        if (x[n] == 0) & (n == 0):
            y.append(1) # if it starts in peace
        elif x[n] == 1:
            y.append(0) # reset to 0 if conflict
        else:
            y.append(y[n-1]+1) # add 1 if peace
    return y

In [ ]:
# add columns that count the months since the last time a given type of conflict was present in that country
cols = [conflict_choice] # could have several conlfict cut-offs (e.g. Hannes has: 'anyviolence', 'armedconf', 'civilwar')
names = [x+'_since' for x in cols] # sincelast variables are called anviolence_dp etc.
df_past[names] = (df_past.groupby('isocode')[cols].transform(count_peace))

In [ ]:
country_df = df_past[df_past['Country Name'] == 'Afghanistan']
country_df[['Country Name', 'month_year', deaths_choice, 'past6', 'past12', 'past60', 'past120', conflict_choice, f'{conflict_choice}_since']]

In [ ]:
def stock_simple(x: pd.Series, decay: float = 0.8):
    # xs is a stock of x inflow with a decay of 0.8
    # nans = x.isnull()
    x = list(x.fillna(0))
    xs = [] 
    for n in range(len(x)):
        if n == 0: 
            xs.append(x[n]) # stock starts in initial value
        else:
            xs.append(x[n] + decay * xs[n-1])
    
    return xs

In [ ]:
df_past[deaths_choice].isna().sum()

In [ ]:
decay = 0.8

unit_of_analyis = 'isocode'

print(df_past.shape)


lcols_novs = (df_past.groupby(unit_of_analyis)
         .apply(lambda x: stock_simple(x[deaths_choice], decay=decay))
         .explode().reset_index(drop=True)
         .rename(f'deaths_stock') for t in range(1,2))


temp_df = pd.DataFrame(lcols_novs).transpose()

print(temp_df.shape)

df_past.sort_values(by=['isocode', 'month_year'], inplace=True)
#temp_df

df_past.reset_index(drop=True, inplace=True)
df_past = df_past.join(temp_df)

print(df_past.shape)

df_past.isna().sum().sum()


In [ ]:
df_past.loc[df_past['isocode'] =='AFG', ['month_year', deaths_choice, 'deaths_stock']]

In [ ]:
df_past.sort_values(by=['month_year', 'isocode'], inplace=True)

## Generate Event shares & normalised total events
We also make an assumption for missing values that allows us to impute them all with zeros:

Missing GDELT event counts: means nothing happened in that country in that time period that was of sufficiently high profile to be captured by GDELT.

In [ ]:
# choose which population data to use
df_shares = df_past.copy() 

In [ ]:
print(df_shares.isnull().sum().sum())

# fill missing events with with 0
#event_count_columns= df_shares.filter(like='event_count').columns.tolist()
event_count_columns = [col for col in df_shares.columns if col.startswith('count_events')]
df_shares[event_count_columns] = df_shares[event_count_columns].fillna(0)

# check all missing values are gone
df_shares.isnull().sum().sum()

In [ ]:
# List of all event count columns for gov, opp and total events
event_cols = ['count_events_{}'.format(i) for i in range(1, 21)]
#event_cols_gov = ['count_events_{}_gov'.format(i) for i in range(1, 21)]
#event_cols_opp = ['count_events_{}_opp'.format(i) for i in range(1, 21)]

# Compute the total events for each group
df_shares['total_events'] = df_shares[event_cols].sum(axis=1)
#df_shares['total_events_gov'] = df_shares[event_cols_gov].sum(axis=1)
#df_shares['total_events_opp'] = df_shares[event_cols_opp].sum(axis=1)

# Check if will be diving by zero
print('min total events:', df_shares['total_events'].min())
#print('min total events gov:', df_shares['total_events_gov'].min())
#print('min total events opp:', df_shares['total_events_opp'].min())

# Compute the share of each type of event for each group and create new columns
for col in event_cols:
    df_shares['share_events_{}'.format(col)] = df_shares[col] / df_shares['total_events'] *100
    
# for col in event_cols_gov:
#     df_shares['share_events_{}'.format(col)] = df_shares[col] / df_shares['total_events_gov'] *100

# for col in event_cols_opp:
#     df_shares['share_events_{}'.format(col)] = df_shares[col] / df_shares['total_events_opp'] *100

# Fill missing values with 0
print('missing values in shares to be filled:', df_shares.isnull().sum().sum())
df_shares = df_shares.fillna(0)

df_shares.isnull().sum().sum()


In [ ]:
for i in range(0, 21):
    old_column_name = 'share_events_count_events_{}'.format(i)
    new_column_name = 'share_events_{}'.format(i)
    # old_column_name_gov = 'share_events_count_events_{}_gov'.format(i)
    # new_column_name_gov = 'share_events_{}_gov'.format(i)
    # old_column_name_opp = 'share_events_count_events_{}_opp'.format(i)
    # new_column_name_opp = 'share_events_{}_opp'.format(i)
    df_shares.rename(columns={old_column_name: new_column_name}, inplace=True) # , old_column_name_gov: new_column_name_gov, old_column_name_opp: new_column_name_opp


INSTEAD of geenrating shares for each _gov and _opp event, we just capture what percentage of events in that month involve _gov and _opp as an actor.

In [ ]:
#specific_events = ['state_deaths', 'nonstate_deaths', 'onesided_deaths', 'civilian_deaths']

event_cols_gov = ['count_events_{}_gov'.format(i) for i in range(1, 21)]
event_cols_opp = ['count_events_{}_opp'.format(i) for i in range(1, 21)]

df_shares['events_gov'] = df_shares[event_cols_gov].sum(axis=1)
df_shares['events_opp'] = df_shares[event_cols_opp].sum(axis=1)

specific_events = ['events_gov', 'events_opp']

# Compute the share of each subset of events of the total events - only when not dividing by zero
for col in specific_events:
    df_shares['share_{}'.format(col)] = np.where((df_shares['total_events'] > 0) & (df_shares[col] > 0),
                                                 round(df_shares[col] / df_shares['total_events'] * 100, 2), 0)

# Drop the original columns
df_shares.drop(columns=specific_events, inplace=True)

df_shares.isnull().sum().sum()

In [ ]:
print(df_shares.share_events_gov.min(), df_shares.share_events_gov.max(), df_shares.share_events_opp.min(), df_shares.share_events_opp.max())

In [ ]:
# just to check the newly generated share columns sum to 100 (NB: some seem to sum to less that 99.5, possibly due to being floats)
new_column_names = ['share_events_{}'.format(i) for i in range(1, 21)]

df_shares['sum_share'] = df_shares[new_column_names].sum(axis=1).astype(int)
print(df_shares['sum_share'].value_counts())

# drop the sum column
df_shares = df_shares.drop(columns=['sum_share'])


In [ ]:
# normalise the events (dividing by the yearly totals)

# plot yearly totals
monthly_totals = df_shares.groupby('month_year')['total_events'].sum()
monthly_totals.plot.bar(figsize=(20, 4), title='Monthly totals of GDELT events')

# normalise the events (dividing by the yearly totals)
df_shares['norm_total_events'] = df_shares.groupby('month_year')['total_events'].transform(lambda x: x / monthly_totals[x.name])

In [ ]:
example_date = '2020-02-01'
afg_jan_2020 = df_shares.loc[(df_shares['isocode'] == 'AFG') & (df_shares['month_year'] == example_date)]['total_events'].astype(int)
all_jan_2020 = monthly_totals[example_date].astype(int)

# print('Total events in Afghanistan in January 2020:', afg_jan_2020)
# print('Total events in all countries in January 2020:', all_jan_2020)

print('Manually normalised total events', afg_jan_2020 / all_jan_2020)
print('Normalised total events from dataframe:')
df_shares.loc[(df_shares['isocode'] == 'AFG') & (df_shares['month_year'] == example_date)]['norm_total_events']

In [ ]:
# Drop the original count_events_* columns and total event columns
df_shares.drop(columns=event_cols + event_cols_gov + event_cols_opp, inplace=True)
df_shares.drop(columns=['total_events'], inplace=True) #'events_gov', 'events_opp'

## Generate stocks of events

In [ ]:
df_stocks = df_shares.copy()

df_stocks.shape

In [ ]:
# Create example data frame with a single column of twos
df = pd.DataFrame({'twos': [2] * 10})

# Pass the 'twos' column to the stock_simple function with decay=0.8
result = stock_simple(df['twos'], decay=0.8)

# Print the result
print(result)


In [ ]:
#event share stocks
decay = 0.8

unit_of_analyis = 'isocode'

print(df_stocks.shape)

lcols_novs = (df_stocks.groupby(unit_of_analyis)
        .apply(lambda x: stock_simple(x[f'share_events_{t}'], decay=decay))
        .explode().reset_index(drop=True)
        .rename(f'event_share_{t}_stock') for t in range(1,21))

new_df = pd.DataFrame(lcols_novs).transpose()

print(new_df.shape)


# sort by country and month to join accurately!
df_stocks.sort_values(by=[unit_of_analyis, 'month_year'], inplace=True)

df_stocks.reset_index(drop=True, inplace=True)

df_stocks = df_stocks.join(new_df)

# sort back to original order
#df_stocks.sort_values(by=[unit_of_analyis, 'month_year'], inplace=True)


print(df_stocks.shape)

new_df.isna().sum().sum()


In [ ]:
df_stocks.loc[df_stocks['isocode'] == 'AFG'][['month_year', 'share_events_1', 'event_share_1_stock']]

## Admin1 features

Decide between which years to target encode for!


In [ ]:
start_encode = 2000
end_encode = 2017 # i.e. train from 2018

In [ ]:
df_final = df_stocks.copy()
adm1 = pd.read_csv(parent_path + "/data/final_gdelt_bycountry.txt")

In [ ]:
# Dropping columns before 1989 to match the GDELT & UCDP data
adm1.drop(adm1.loc[adm1['year']<start_encode].index, inplace=True)


In [ ]:
adm1.sort_values(by=["isocode","year","month","ActionGeo_ADM1Code"])

number_regions = adm1.groupby(["isocode", "year", "month"]).nunique()["ActionGeo_ADM1Code"].reset_index()
number_regions = number_regions.rename({"ActionGeo_ADM1Code": "num_regions"}, axis=1)
number_regions.head(10)

In [ ]:
# Merging with the full dataset
df_final['month'] = df_final['month'].astype(int)
df_final = pd.merge(df_final, number_regions, on= ["isocode", "year", "month"], how = "left") # left_on=["isocode", "year", "month"], right_on = ["isocode", "year", "month"]


# check no zeros
(df_final.num_regions.isna().sum()) / len(df_final) 

In [ ]:
# Dropping adm1 that does not have relevant events
# The 'fight' column will contain 1 if there's a non-zero value in any of the columns, otherwise 0

pre_drop = len(adm1)
print('adm1 frame pre drop', pre_drop)

adm1['fight'] = ((adm1['count_events_18'].fillna(0) > 0) | 
                        (adm1['count_events_19'].fillna(0) > 0) | 
                        (adm1['count_events_20'].fillna(0) > 0)).astype(int)

adm1 = adm1[adm1['fight'] == 1] # This will keep only the rows where 'fight' column is equal to 1

post_drop = len(adm1)
print('adm1 frame post drop', post_drop)
print('droped:', pre_drop - post_drop)

In [ ]:
# adm1['year'] = adm1['year'].astype(int)
# adm1['month'] = adm1['month'].astype(int)

adm_full = pd.merge(df_final, adm1[['ActionGeo_ADM1Code',"isocode", "month", "year"]], 
                               left_on=["isocode", "month", "year"], right_on = ["isocode", "month", "year"], 
                               how = "left")


In [ ]:
adm_full.head(10)

In [ ]:
def encode_adm(df, year_threshold):
    # Split the data into train and test based on the year threshold
    df_train = df[df['year'] <= year_threshold]
    df_test = df[df['year'] > year_threshold]

    # Target encoding the regions that have at least 1 event of type 18, 19, or 20
    region_encoder = ce.TargetEncoder(smoothing=1.0)
    region_encoder.fit(df_train['ActionGeo_ADM1Code'], df_train[deaths_choice]) #['deaths'])
    df_train['ActionGeo_ADM1Code'] = region_encoder.transform(df_train['ActionGeo_ADM1Code'], df_train[deaths_choice]) #['deaths'])
    df_test['ActionGeo_ADM1Code'] = region_encoder.transform(df_test['ActionGeo_ADM1Code'], df_test[deaths_choice]) #['deaths'])
    df = pd.concat([df_train, df_test])

    # Getting the maximum, mean, and median regions for each month/year and country
    df['Adm1_Max'] = df.groupby(['isocode', 'month', 'year'])['ActionGeo_ADM1Code'].transform('max')
    df['Adm1_Mean'] = df.groupby(['isocode', 'month', 'year'])['ActionGeo_ADM1Code'].transform('mean')
    df['Adm1_Median'] = df.groupby(['isocode', 'month', 'year'])['ActionGeo_ADM1Code'].transform('median')

    df_transform = df.drop_duplicates(['isocode', 'month', 'year']).drop(["ActionGeo_ADM1Code"], axis=1)

    return df_transform


In [ ]:
df_admin = encode_adm(adm_full, end_encode) #only keeps years after in train

## Share of death types

In [ ]:
df_deaths = df_admin.copy()

print('min total events:', df_deaths['deaths'].min())

In [ ]:
specific_deaths = ['state_deaths', 'nonstate_deaths', 'onesided_deaths', 'civilian_deaths']

# Compute the share of each subset of deaths of the total deaths - only when not dividing by zero
for col in specific_deaths:
    df_deaths['share_{}'.format(col)] = np.where((df_deaths['deaths'] > 0) & (df_deaths[col] > 0),
                                                 round(df_deaths[col] / df_deaths['deaths'] * 100, 2), 0)

# Drop the original columns
df_deaths.drop(columns=specific_deaths, inplace=True)

df_deaths.isnull().sum().sum()

In [ ]:
# option to scroll through all columns generated after merging population data
print((set(df_deaths.columns) - set(pop_options.columns)))

## Cyclical months feature

In [ ]:
df_cyclical = df_deaths.copy()

encoder = CyclicalFeatures(variables=['month'], drop_original=False)
df_cyclical = encoder.fit_transform(df_cyclical)

df_cyclical.loc[df_cyclical['isocode'] == 'ALB', ['month_year', 'month_sin', 'month_cos']]

## Refugee data

In [ ]:
df_refugees = df_cyclical.copy()

In [ ]:
def interpolate_group(group):
    group[columns_to_interpolate] = group[columns_to_interpolate].interpolate(method='linear')
    return group

def add_refugee_flows(preprocessed_df):
    """
    Add refugee flows within each country group and interpolate by month.

    Args:
        preprocessed_df (pandas.DataFrame): DataFrame containing the preprocessed data.

    Returns:
        pandas.DataFrame: DataFrame with interpolated refugee flows.

    """
    
    # Read UNHCR data
    df = pd.read_csv(os.path.abspath(parent_path + '/data/UNHCR.csv'))

    # Create 'Total' column
    df['Total'] = df.iloc[:, 5:9].sum(axis=1)

    # Retrieving the list of countries of origin and destination 
    common_countries_origin = set(df['Country of origin (ISO)']).intersection(set(preprocessed_df['isocode']))
    common_countries_destination = set(df['Country of asylum (ISO)']).intersection(set(preprocessed_df['isocode']))

    # Aggregating refugee flows at country of origin level
    df_origin_agg = df.groupby(['Country of origin (ISO)', 'Year'])['Total'].sum().reset_index()
    df_origin_agg.rename(columns={'Total': 'refugees_out'}, inplace=True)

    # Aggregating refugee flows at country of destination level
    df_destination_agg = df.groupby(['Country of asylum (ISO)', 'Year'])['Total'].sum().reset_index()
    df_destination_agg.rename(columns={'Total': 'refugees_in'}, inplace=True)

    # Merge aggregated refugee flows with prepross
    df_wref = preprocessed_df.merge(df_origin_agg, left_on=['isocode', 'year'], right_on=['Country of origin (ISO)', 'Year'], how='left')
    df_wref = df_wref.merge(df_destination_agg, left_on=['isocode', 'year'], right_on=['Country of asylum (ISO)', 'Year'], how='left')
    df_wref.drop(['Country of origin (ISO)', 'Country of asylum (ISO)', 'Year_x', 'Year_y'], axis=1, inplace=True)

    # Fill NAs before 2023 with 0
    df_wref.loc[df_wref['year'] < 2023, ['refugees_out', 'refugees_in']] = \
        df_wref.loc[df_wref['year'] < 2023, ['refugees_out', 'refugees_in']].fillna(0)

    # Set values to NA for non-January months
    df_wref['month_year'] = pd.to_datetime(df_wref['month_year'], format='%Y-%m-%d')
    df_wref.loc[df_wref['month_year'].dt.month != 1, 'refugees_out'] = np.nan
    df_wref.loc[df_wref['month_year'].dt.month != 1, 'refugees_in'] = np.nan

    # Set 'isocode' and 'month_year' as the index
    df_wref = df_wref.set_index(['isocode', 'month_year'])

    # List of columns to interpolate
    columns_to_interpolate = ['refugees_out', 'refugees_in']

    # Apply interpolation within each country group
    df_wref = df_wref.groupby('isocode').apply(interpolate_group)

    # Reset the index
    df_wref = df_wref.reset_index(drop=False)

    return df_wref

In [ ]:
columns_to_interpolate = ['refugees_out', 'refugees_in']

df_refugees = add_refugee_flows(df_refugees)
print(df_refugees.isnull().sum().sum())
df_refugees.head(2)

In [ ]:
df_refugees['refugees_out'] = df_refugees['refugees_out'] / df_refugees[pop_choice]
df_refugees['refugees_in'] = df_refugees['refugees_in'] / df_refugees[pop_choice]

df_refugees.head(2)

In [ ]:
# all missing values are for the Maldives as they only appear in GDELT from 2000 Feb 
#filtered_df = df_refugees[df_refugees.isnull().any(axis=1)]
#filtered_df
refugee_nulls = ['refugees_out', 'refugees_in']
df_refugees[refugee_nulls] = df_refugees[refugee_nulls].fillna(0)

print(df_refugees.isnull().sum().sum())



## Neighbouring countries

In [ ]:
df_neighb_og = df_refugees.copy() #df_cyclical.copy()
df_neighb_og.columns

In [ ]:
isocode_dict = {
    'AFG': ['IRN', 'PAK', 'TKM', 'UZB', 'TJK', 'CHN'],
    'AGO': ['COG', 'COD', 'ZMB', 'NAM'],
    'ALB': ['GRC', 'MKD', 'MNE', 'SRB'],
    'ARE': ['OMN', 'SAU'],
    'ARG': ['BOL', 'BRA', 'CHL', 'PRY', 'URY'],
    'ARM': ['AZE', 'GEO', 'IRN', 'TUR'],
    'ATG': [],
    'AUS': [],
    'AUT': ['CZE', 'DEU', 'HUN', 'ITA', 'LIE', 'SVK', 'SVN', 'CHE'],
    'AZE': ['ARM', 'GEO', 'IRN', 'RUS', 'TUR'],
    'BDI': ['COD', 'RWA', 'TZA'],
    'BEL': ['FRA', 'DEU', 'LUX', 'NLD'],
    'BEN': ['BFA', 'NER', 'NGA', 'TGO'],
    'BFA': ['BEN', 'CIV', 'GHA', 'MLI', 'NER', 'TGO'],
    'BGD': ['MMR', 'IND'],
    'BGR': ['GRC', 'MKD', 'ROU', 'SRB', 'TUR'],
    'BHR': ['KWT', 'SAU'],
    'BHS': [],
    'BIH': ['HRV', 'MNE', 'SRB'],
    'BLR': ['LVA', 'LTU', 'POL', 'RUS', 'UKR'],
    'BLZ': ['GTM', 'MEX'],
    'BMU': [],
    'BOL': ['ARG', 'BRA', 'CHL', 'PRY', 'PER'],
    'BRA': ['ARG', 'BOL', 'COL', 'GUF', 'GUY', 'PRY', 'PER', 'SUR', 'URY', 'VEN'],
    'BRB': [],
    'BRN': ['IDN', 'MYS'],
    'BTN': ['CHN', 'IND'],
    'BWA': ['NAM', 'ZAF', 'ZWE'],
    'CAF': ['CMR', 'COD', 'SDN', 'SSD', 'TCD'],
    'CAN': ['USA'],
    'CHE': ['AUT', 'DEU', 'FRA', 'ITA', 'LIE'],
    'CHL': ['ARG', 'BOL', 'PER'],
    'CHN': ['AFG', 'BTN', 'HKG', 'IND', 'KAZ', 'KGZ', 'LAO', 'MAC', 'MNG', 'MMR', 'NPL', 'PRK', 'RUS', 'TJK', 'VNM'],
    'CIV': ['BFA', 'GHA', 'GIN', 'LBR', 'MLI'],
    'CMR': ['CAF', 'CAF', 'COD', 'GNQ', 'NGA', 'TCD'],
    'COD': ['AGO', 'BDI', 'CAF', 'COG', 'RWA', 'SSD', 'TZA', 'UGA', 'ZMB'],
    'COG': ['AGO', 'CAF', 'COD', 'GAB'],
    'COL': ['BRA', 'ECU', 'PAN', 'PER', 'VEN'],
    'COM': [],
    'CPV': [],
    'CRI': ['NIC', 'PAN'],
    'CUB': [],
    'CYP': [],
    'CZE': ['AUT', 'DEU', 'POL', 'SVK'],
    'DEU': ['AUT', 'BEL', 'CZE', 'DNK', 'FRA', 'LUX', 'NLD', 'POL', 'CHE'],
    'DJI': ['ETH', 'ERI', 'SOM'],
    'DMA': [],
    'DNK': ['DEU'],
    'DOM': ['HTI'],
    'DZA': ['LBY', 'MAR', 'MRT', 'NER', 'TUN', 'ESH'],
    'ECU': ['COL', 'PER'],
    'EGY': ['ISR', 'LBY', 'SDN'],
    'ERI': ['DJI', 'ETH', 'SDN'],
    'ESH': ['DZA', 'MRT', 'MAR'],
    'ESP': ['AND', 'FRA', 'GIB', 'MAR', 'PRT'],
    'EST': ['LVA', 'RUS'],
    'ETH': ['DJI', 'ERI', 'KEN', 'SDN', 'SOM', 'SSD'],
    'FIN': ['NOR', 'RUS', 'SWE'],
    'FJI': [],
    'FRA': ['AND', 'BEL', 'DEU', 'ESP', 'ITA', 'LUX', 'MCO', 'CHE'],
    'FSM': [],
    'GAB': ['CMR', 'COG', 'GNQ'],
    'GBR': [],
    'GEO': ['ARM', 'AZE', 'RUS', 'TUR'],
    'GHA': ['BFA', 'CIV', 'TGO'],
    'GIB': ['ESP'],
    'GIN': ['CIV', 'GNB', 'LBR', 'MLI', 'SEN', 'SLE'],
    'GMB': ['SEN'],
    'GNB': ['GIN', 'SEN'],
    'GNQ': ['CMR', 'GAB'],
    'GRC': ['ALB', 'BGR', 'TUR'],
    'GRD': [],
    'GRL': [],
    'GTM': ['BLZ', 'HND', 'MEX', 'SLV'],
    'GUM': [],
    'GUY': ['BRA', 'SUR', 'VEN'],
    'HKG': ['CHN'],
    'HND': ['GTM', 'NIC', 'SLV'],
    'HRV': ['BIH', 'HUN', 'MNE', 'SRB', 'SVN'],
    'HTI': ['DOM'],
    'HUN': ['AUT', 'HRV', 'ROU', 'SRB', 'SVK', 'SVN', 'UKR'],
    'IDN': ['BRN', 'MYS', 'PNG', 'TLS'],
    'IND': ['AFG', 'BGD', 'BTN', 'CHN', 'MMR', 'NPL', 'PAK'],
    'IRL': ['GBR'],
    'IRN': ['AFG', 'ARM', 'AZE', 'IRQ', 'PAK', 'TUR', 'TKM'],
    'IRQ': ['IRN', 'JOR', 'KWT', 'SAU', 'SYR', 'TUR'],
    'ISL': [],
    'ISR': ['EGY', 'JOR', 'LBN', 'PSE', 'SYR'],
    'ITA': ['AUT', 'FRA', 'SMR', 'SVN', 'CHE', 'VAT'],
    'JAM': [],
    'JOR': ['IRQ', 'ISR', 'SAU', 'SYR'],
    'JPN': [],
    'KAZ': ['CHN', 'KGZ', 'RUS', 'TKM', 'UZB'],
    'KEN': ['ETH', 'SOM', 'SSD', 'TZA', 'UGA'],
    'KGZ': ['CHN', 'KAZ', 'TJK', 'UZB'],
    'KHM': ['LAO', 'THA', 'VNM'],
    'KOR': ['PRK'],
    'KWT': ['IRQ', 'SAU'],
    'LAO': ['CHN', 'KHM', 'MMR', 'THA', 'VNM'],
    'LBN': ['ISR', 'SYR'],
    'LBR': ['GIN', 'CIV', 'GNB', 'SLE'],
    'LBY': ['DZA', 'EGY', 'NER', 'SDN', 'TCD', 'TUN'],
    'LKA': ['IND'],
    'LSO': ['ZAF'],
    'LTU': ['BLR', 'LVA', 'POL', 'RUS'],
    'LUX': ['BEL', 'DEU', 'FRA'],
    'LVA': ['BLR', 'EST', 'LTU', 'RUS'],
    'MAC': ['CHN'],
    'MAR': ['DZA', 'ESH', 'ESP'],
    'MCO': ['FRA'],
    'MDA': ['ROU', 'UKR'],
    'MDG': [],
    'MDV': [],
    'MEX': ['BLZ', 'GTM', 'USA'],
    'MHL': [],
    'MKD': ['ALB', 'BGR', 'GRC', 'SRB'],
    'MLI': ['BFA', 'GIN', 'NER', 'SEN'],
    'MLT': [],
    'MMR': ['BGD', 'CHN', 'IND', 'LAO', 'THA'],
    'MNE': ['ALB', 'BIH', 'HRV', 'SRB'],
    'MNG': ['CHN', 'RUS'],
    'MNP': [],
    'MOZ': ['MWI', 'ZAF', 'SWZ', 'TZA', 'ZMB', 'ZWE'],
    'MRT': ['DZA', 'MLI', 'SEN'],
    'MUS': [],
    'MWI': ['MOZ', 'TZA', 'ZMB'],
    'MYS': ['BRN', 'IDN', 'THA'],
    'NAM': ['AGO', 'BWA', 'ZAF', 'ZMB'],
    'NCL': [],
    'NER': ['BEN', 'DZA', 'LBY', 'MLI', 'NGA', 'TCD'],
    'NGA': ['BEN', 'CMR', 'NER'],
    'NIC': ['CRI', 'HND'],
    'NLD': ['BEL', 'DEU'],
    'NOR': ['FIN', 'RUS', 'SWE'],
    'NPL': ['CHN', 'IND'],
    'NZL': [],
    'OMN': ['ARE', 'SAU', 'YEM'],
    'PAK': ['AFG', 'CHN', 'IND', 'IRN'],
    'PAN': ['COL', 'CRI'],
    'PER': ['BOL', 'BRA', 'CHL', 'COL', 'ECU'],
    'PHL': [],
    'PNG': ['IDN'],
    'POL': ['BLR', 'CZE', 'DEU', 'LTU', 'RUS', 'SVK', 'UKR'],
    'PRI': [],
    'PRK': ['CHN', 'KOR', 'RUS'],
    'PRT': ['ESP'],
    'PRY': ['ARG', 'BOL', 'BRA'],
    'PSE': ['ISR', 'JOR'],
    'QAT': ['SAU'],
    'ROU': ['BGR', 'HUN', 'MDA', 'SRB', 'UKR'],
    'RUS': ['AZE', 'BLR', 'EST', 'FIN', 'GEO', 'KAZ', 'LVA', 'LTU', 'MNG', 'NOR', 'POL', 'UKR'],
    'RWA': ['BDI', 'COD', 'TZA', 'UGA'],
    'SAU': ['ARE', 'IRQ', 'JOR', 'KWT', 'OMN', 'QAT', 'UAE', 'YEM'],
    'SDN': ['CAF', 'EGY', 'ERI', 'ETH', 'LBY', 'SSD', 'TCD'],
    'SEN': ['GIN', 'GNB', 'MRT'],
    'SGP': [],
    'SLB': [],
    'SLE': ['GIN', 'LBR'],
    'SLV': ['GTM', 'HND'],
    'SOM': ['DJI', 'ETH', 'KEN'],
    'SRB': ['ALB', 'BIH', 'BGR', 'HRV', 'HUN', 'KOS', 'MKD', 'MNE', 'ROU'],
    'SSD': ['CAF', 'COD', 'ETH', 'KEN', 'SDN', 'UGA'],
    'STP': [],
    'SUR': ['BRA', 'GUF', 'GUY'],
    'SVK': ['AUT', 'CZE', 'HUN', 'POL', 'UKR'],
    'SVN': ['AUT', 'HRV', 'HUN', 'ITA'],
    'SWE': ['FIN', 'NOR'],
    'SWZ': ['MOZ', 'ZAF'],
    'SYC': [],
    'SYR': ['IRQ', 'ISR', 'JOR', 'LBN', 'TUR'],
    'TCD': ['CMR', 'LBY', 'NER', 'SDN'],
    'TGO': ['BEN', 'BFA', 'BFA', 'GHA'],
    'THA': ['KHM', 'LAO', 'MYS', 'MMR'],
    'TJK': ['AFG', 'CHN', 'KGZ', 'UZB'],
    'TKM': ['AFG', 'IRN', 'KAZ', 'UZB'],
    'TLS': ['IDN'],
    'TON': [],
    'TTO': [],
    'TUN': ['DZA', 'LBY'],
    'TUR': ['ARM', 'BGR', 'GEO', 'GRC', 'IRN', 'IRQ', 'SYR'],
    'TWN': [],
    'TZA': ['BDI', 'COD', 'KEN', 'MWI', 'MOZ', 'RWA', 'UGA', 'ZMB'],
    'UGA': ['COD', 'KEN', 'RWA', 'SSD', 'TZA'],
    'UKR': ['BLR', 'HUN', 'MDA', 'POL', 'ROU', 'RUS', 'SVK'],
    'URY': ['ARG', 'BRA'],
    'USA': ['CAN', 'MEX'],
    'UZB': ['AFG', 'KAZ', 'KGZ', 'TJK', 'TKM'],
    'VEN': ['BRA', 'COL', 'GUY'],
    'VIR': [],
    'VNM': ['CHN', 'KHM', 'LAO'],
    'VUT': [],
    'WSM': [],
    'XKX': ['ALB', 'MNE', 'MKD', 'SRB'],
    'YEM': ['OMN', 'SAU'],
    'ZAF': ['BWA', 'LSO', 'MOZ', 'NAM', 'SWZ', 'ZWE'],
    'ZMB': ['AGO', 'MOZ', 'MWI', 'NAM', 'TZA', 'ZWE'],
    'ZWE': ['MOZ', 'ZAF', 'ZMB']}


In [ ]:
#df = pd.read_csv(os.path.abspath(parent_path + '/data/preprocessed_df.csv'))

df_neighb = df_neighb_og.merge(pd.DataFrame(isocode_dict.items(), columns=['isocode', 'neighbors']), on='isocode', how='left')

def check_neighbor_conflict(row, df):
    """
    Check if at least one neighbor is in conflict based on the row of the dataframe.

    Args:
        row (pd.Series): Row of the dataframe containing country information.
        df (pd.DataFrame): The entire dataframe containing country and neighbor information.

    Returns:
        bool: True if at least one neighbor is in conflict, False otherwise.
    """
    neighbors = row['neighbors']
    if isinstance(neighbors, list):
        neighbor_status = df[
            (df['isocode'].isin(neighbors)) &
            (df['month_year'] == row['month_year'])
        ][conflict_choice]
        if len(neighbor_status) > 0 and neighbor_status.any():
            return True
    return False



In [ ]:
# Apply the function row-wise to create the new column
df_neighb['neighbor_conflict'] = df_neighb.apply(check_neighbor_conflict, axis=1, df=df_neighb)
df_neighb.head(2)

## Clusters

In [ ]:
df_clusters = df_neighb.copy()

# ensure we only encode the clusters on the basis of the training data
train_end

In [ ]:
df_clusters_train = df_clusters.loc[df_clusters['year'] <= train_end]

print(df_clusters_train.month_year.min(), df_clusters_train.month_year.max())

## group columns

event_share_cols = df_clusters_train.filter(like='share_events').columns.tolist()  #event_share_cols = ['share_events_{}'.format(i) for i in range(1, 21)] 
event_stock_cols = ['event_share_{}_stock'.format(i) for i in range(1, 21)] 
#death_share_cols = df_clusters.filter(like='_deaths').columns.tolist() # state, nonstate, one-sided, civilians
#past_cols = ['deaths_stock'] #, 'past6', 'past12',	'past60', 'past120']

#peace_count = conflict_choice + '_since'
#adm1_cols = ['Adm1_Median', 'Adm1_Mean']

# how to aggregate them
mean_max =  [deaths_choice]   #+ death_share_cols  + adm1_cols + + [peace_count] + ['deaths'] + past_cols 
#max_only = ['num_regions'] + ['Adm1_Max']
mean_only = [conflict_choice] + ['norm_total_events'] #+ ['Adm1_Mean'] 
median_only = event_stock_cols
all_aggs = event_share_cols

cols_not_used = set(df_clusters_train.columns.tolist()) - set(mean_max + max_only + mean_only + median_only + all_aggs)
cols_not_used

In [ ]:
# Empty dictionaries to store the aggregation functions, which are dynamically generated for each feature column
agg_functions_mean_max = {}
agg_functions_max_only = {}
agg_functions_mean_only = {}
agg_functions_median_only = {}
agg_functions_all_aggs = {}

for column in mean_max:
    agg_functions_mean_max[column] = ['mean', 'max'] #median

# for column in max_only:
#     agg_functions_max_only[column] = ['max']

for column in mean_only:
    agg_functions_mean_only[column] = ['mean']

for column in all_aggs:
    agg_functions_median_only[column] = ['median']

for column in all_aggs:
    agg_functions_all_aggs[column] = ['mean', 'min', 'max']

In [ ]:
aggregated_data = (df_clusters_train.groupby(unit_of_analyis)
                .agg({**agg_functions_mean_max,
                      #**agg_functions_max_only,
                      **agg_functions_mean_only,
                      **agg_functions_median_only,
                      **agg_functions_all_aggs}))


Principal Component Analyis to reduce dimensionality.

In [ ]:
X = aggregated_data.copy()
country_labels = X.index

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA()
X_pca = pca.fit_transform(X_scaled)

explained_variance_ratio = pca.explained_variance_ratio_
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.show()

n_components = 3  # Set the desired number of components
X_selected = X_pca[:, :n_components]

Clusters based on principal components.

In [ ]:
pca_df = pd.DataFrame(X_selected, columns=['PC1', 'PC2', 'PC3'])  # Adjust column names as needed
pca_df['country'] = country_labels  # Add the country labels column
pca_df.set_index('country', inplace=True)  # Set the country column as the index
pca_df

In [ ]:
# slelect columns for clustering
data_pca = pca_df

# normalise data
scaler = StandardScaler()
normalized_data_pca = scaler.fit_transform(data_pca)

# pick cluster range
k_values = [5,10,15,20,30, 40] #range(1, 20)

# initialize list to store inertia values
inertia_values = []

#perform k-means clustering for each value of k
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=1000, init = 'k-means++')
    kmeans.fit(normalized_data_pca)
    inertia_values.append(kmeans.inertia_)

# plot the elbow curve
plt.plot(k_values, inertia_values, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Curve')
plt.show()

In [ ]:
# Determine the optimal number of clusters based on the elbow point
optimal_k = 15

# Perform k-means clustering with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10, max_iter=1000, init = 'k-means++')
kmeans.fit(normalized_data_pca)

# Assign observations to clusters
cluster_labels = kmeans.labels_

# Add cluster labels to the dataframe
data_pca['cluster'] = cluster_labels

# Function to get country name from ISO code
def get_country_name(iso_code):
    try:
        country = pc.countries.get(alpha_3=iso_code)
        return country.name
    except AttributeError:
        return 'Unknown'

# Generate column of country names
data_pca['isocode'] = data_pca.index
data_pca['Country Name'] = data_pca['isocode'].apply(get_country_name)

## Clusters based on PCA version of aggregated features
# cluster_names = []
# for i in range(0,15):
#     print(f'cluster {i}' , data_pca.loc[data_pca['Cluster'] == i].index.tolist())

cluster_names = []
for i in range(0,15):
    print(f'cluster {i}' , data_pca.loc[data_pca['cluster'] == i, 'Country Name'].tolist())


# save values in a dictionary to apply to the original data
cluster_dict = data_pca.groupby('cluster')[unit_of_analyis].apply(list).to_dict()
print('cluster dictionary example', cluster_dict[0])

In [ ]:
# Create a color palette with 15 distinct colors
num_clusters = data_pca['cluster'].nunique()
palette = sns.color_palette('hsv', num_clusters)

fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection='3d')

for index, row in data_pca.iterrows():
    cluster_color = palette[row['cluster'] - 1]
    ax.scatter(row['PC1'], row['PC2'], row['PC3'], c=[cluster_color], label=row[unit_of_analyis])
    ax.text(row['PC1'], row['PC2'], row['PC3'], row[unit_of_analyis], color='black', fontsize=10, ha='center', va='center')

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

legend_elements = []
for cluster_label, color in enumerate(palette, start=1):
    legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, label=f'Cluster {cluster_label}'))

# Place the legend outside the graph
ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()


In [ ]:
df_clusters_joined = pd.merge(df_clusters, data_pca[[unit_of_analyis, 'cluster']], on= unit_of_analyis, how='left')

print(df_clusters_joined.shape, df_clusters.shape, df_clusters_train.shape)

In [ ]:
print(df_clusters_joined.shape, df_clusters.shape)
df_clusters_joined.head()

In [ ]:
ce_bin = ce.BinaryEncoder(cols = ['cluster'])
X = df_clusters_joined['cluster']
cluster_enc = ce_bin.fit_transform(X)

print(cluster_enc.shape, df_clusters_joined.shape)

df_clusters = pd.concat([df_clusters_joined, cluster_enc], axis=1)
df_clusters.head(1)

## Binary Country Encoder (for tree-based models)

In [ ]:
df_cntry_enc = df_clusters.copy()

ce_bin = ce.BinaryEncoder(cols = [unit_of_analyis])
X = df_cntry_enc[unit_of_analyis]
cntry_enc = ce_bin.fit_transform(X)

print(cntry_enc.shape, df_cntry_enc.shape)

df_cntry_enc = pd.concat([df_cntry_enc, cntry_enc], axis=1)
df_cntry_enc

## Targets within a given period

In [ ]:
df_extra_targets = df_cntry_enc.copy()

In [ ]:
def make_target_regr(df: pd.DataFrame, shifters: dict, target: str):
    """
    Generate shifted variables and calculate the maximum for multiple shifters.

    Args:
        df (pd.DataFrame): The input DataFrame.
        shifters (dict): A dictionary specifying the shifters.
            Example: {3: 'w3_', 6: 'w6_'} for 2 shifters with prefixes 'w3_' and 'w6_'.
        target (str): The column name of the target variable.

    Returns:
        pd.DataFrame: The modified DataFrame with the maximum for each shifter.

    """
    for shifter, prefix in shifters.items():
        # Loop through each period and generate the shift variables
        for i in range(1, shifter + 1):
            col_name = f'{prefix}{target}{i}'
            df[col_name] = df.groupby('isocode')[target].shift(-i)

        # Take the maximum for t periods forward and create the new variable
        avg_col_name = f'{prefix}target_regr'
        df[avg_col_name] = df[[f'{prefix}{target}{i}' for i in range(1, shifter + 1)]].mean(axis=1, skipna=False)

        # Drop the shift variables
        df = df.drop(columns=[f'{prefix}{target}{i}' for i in range(1, shifter + 1)])

        # Shift the resulting column by 1 so that Luis can shift back in LSTM
        df[avg_col_name] = df[avg_col_name].shift(1)

    return df


def make_target_clsf(df: pd.DataFrame, shifters: dict, target: str):
    """
    Generate shifted variables and calculate the maximum for multiple shifters.

    Args:
        df (pd.DataFrame): The input DataFrame.
        shifters (dict): A dictionary specifying the shifters.
            Example: {3: 'w3_', 6: 'w6_'} for 2 shifters with prefixes 'w3_' and 'w6_'.
        target (str): The column name of the target variable.

    Returns:
        pd.DataFrame: The modified DataFrame with the maximum for each shifter.

    """
    for shifter, prefix in shifters.items():
        # Loop through each period and generate the shift variables
        for i in range(1, shifter + 1):
            col_name = f'{prefix}{target}{i}'
            df[col_name] = df.groupby('isocode')[target].shift(-i)

        # Take the maximum for t periods forward and create the new variable
        max_col_name = f'{prefix}target_clsf'
        df[max_col_name] = df[[f'{prefix}{target}{i}' for i in range(1, shifter + 1)]].max(axis=1, skipna=False)

        # Drop the shift variables

        df = df.drop(columns=[f'{prefix}{target}{i}' for i in range(1, shifter + 1)])

        # Shift the resulting column by 1 so that Luis can shift back in LSTM
        df[max_col_name] = df[max_col_name].shift(1)

    return df



In [ ]:
shifters = {3: 'f3_', 6: 'f6_'}


df_extra_targets = make_target_clsf(df_extra_targets, shifters,  'escalation') #'clsf')

df_extra_targets.rename(columns={'f3_target_clsf': 'f3_target_escl', 'f6_target_clsf': 'f6_target_escl'}, inplace=True)

df_extra_targets = make_target_clsf(df_extra_targets, shifters,  conflict_choice) #'clsf')
df_extra_targets = make_target_regr(df_extra_targets, shifters, deaths_choice) #'regr')

df_extra_targets.columns

In [ ]:
df_extra_targets

## Final data check

In [ ]:
df_final = df_extra_targets.copy()

# reminder of chosen variables - then overwrite them with generic versions
print('pop choice:', pop_choice)
print('conflict choice:', conflict_choice)
print('deaths choice:', deaths_choice)

df_final.rename(columns={deaths_choice: 'deaths_all_pc', conflict_choice: 'armedconf', 'armedconf_intp_pop_since': 'armedconf_since'}, inplace=True)

# drop columns we do not use for models
df_final.drop(columns=['country', pop_choice, 'MonthYear', 'deaths', 'neighbors', 'cluster'], inplace=True) #'Country Name'



# Check date range
print(df_final.month_year.min(), df_final.month_year.max())
print('nan', df_final.isna().sum().sum())

df_final.columns

## Export to csv

In [ ]:
filter_df = df_final[df_final.isna().any(axis=1)]
filter_df

In [ ]:
df_final.to_csv(path + "/preprocessed_df.csv", index=False)

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 30)
df_final